# Dual Arm Motion Planning 🦾🦾

## 1. Setup 🏗️️

### 1.1 Recreating the ICRA 2024 Cloth Competition Scene 👕

In [ ]:
import numpy as np
from airo_drake import DualArmScene, add_floor, add_manipulator, add_meshcat, add_wall, finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker


def add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder: RobotDiagramBuilder, arm_left_transform: RigidTransform, arm_right_transform: RigidTransform
) -> tuple[tuple[ModelInstanceIndex, ModelInstanceIndex], tuple[ModelInstanceIndex, ModelInstanceIndex]]:
    parser = robot_diagram_builder.parser()
    parser.SetAutoRenaming(True)

    add_floor(robot_diagram_builder, y_size=2.4)

    # Arms places side by side on the table
    arm_y = arm_left_transform.translation()[1] 

    # Add three safety walls
    wall_thickness = 0.2
    wall_left_position = np.array([0, arm_y + 0.7 + wall_thickness / 2, 0])
    wall_right_position = np.array([0, -arm_y - 0.7 - wall_thickness / 2, 0])
    wall_back_position = np.array([0.9 + wall_thickness / 2, 0, 0])
    add_wall(robot_diagram_builder, "XZ", 2.0, 2.0, wall_thickness, wall_left_position, "wall_left")
    add_wall(robot_diagram_builder, "XZ", 2.0, 2.0, wall_thickness, wall_right_position, "wall_right")
    add_wall(robot_diagram_builder, "YZ", 2.0, 2.7, wall_thickness, wall_back_position, "wall_back")

    # The robot arms
    arm_left_index, gripper_left_index = add_manipulator(
        robot_diagram_builder, "ur5e", "robotiq_2f_85", arm_left_transform, static_gripper=True
    )
    arm_right_index, gripper_right_index = add_manipulator(
        robot_diagram_builder, "ur5e", "robotiq_2f_85", arm_right_transform, static_gripper=True
    )

    return (arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index)

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)

arm_y = 0.45
X_W_L = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, arm_y, 0])
X_W_R = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, -arm_y, 0])


(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_L, X_W_R
)
robot_diagram, context = finish_build(robot_diagram_builder)

scene = DualArmScene(robot_diagram, arm_left_index, arm_right_index, gripper_left_index, gripper_right_index, meshcat)

### 1.2 Dual arm `SceneGraphCollisionChecker` 💥💥

The only diffrence when making a `SceneGraphCollisionChecker` for dual arm is you need to provide it additional `ModelInstanceIndex`s.  

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.arm_left_index,
        scene.arm_right_index,
        scene.gripper_left_index,
        scene.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
scene.robot_diagram.plant().num_positions()

In [ ]:
collision_checker.CheckConfigCollisionFree(np.zeros(12))

In [ ]:
home_joints_left = np.deg2rad([180, -120, 60, -30, -90, -90])
home_joints_right = np.deg2rad([-180, -60, -60, -150, 90, 90])

home_joints = np.concatenate([home_joints_left, home_joints_right])

collision_checker.CheckConfigCollisionFree(home_joints)

In [ ]:
plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
 
plant.SetPositions(plant_context, home_joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

### 1.3 Inverse kinematics for both arms 🧮🧮
We will wrap the individual IK functions to express TCP poses in a common world frame.

In [ ]:
from functools import partial
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from airo_drake import X_URBASE_ROSBASE
from ur_analytic_ik import ur5e

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

def inverse_kinematics_in_world_fn(
    tcp_pose: HomogeneousMatrixType, X_W_CB: HomogeneousMatrixType
) -> list[JointConfigurationType]:
    X_W_TCP = tcp_pose
    X_CB_W = np.linalg.inv(X_W_CB)
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(X_CB_W @ X_W_TCP, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions

# This conversion is needed because the URDFs use the ROS base frame convention:
X_CB_B = X_URBASE_ROSBASE
X_W_LCB = X_W_L.GetAsMatrix4() @ np.linalg.inv(X_CB_B.GetAsMatrix4())
X_W_RCB = X_W_R.GetAsMatrix4() @ np.linalg.inv(X_CB_B.GetAsMatrix4())

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB)

### 1.4 `DualArmOmplPlanner` 🧭

In [ ]:
from airo_planner import DualArmOmplPlanner

joint_bounds_lower = np.deg2rad([-360, -180, -160, -360, -360, -360])
joint_bounds_upper = np.deg2rad([360, 0, 160, 360, 360, 360])
joint_bounds = (joint_bounds_lower, joint_bounds_upper)


planner = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

We use the terms "left" and "right" for the two arms, note that you can choose yourself which arm is which, however you must take care to be consistent which arms configuration is first and which is second in the `is_state_valid_fn`. We recommend for simplificity to choose as left arm the arm that also appear on the left side of image from the robot's primary camera, as this is how an egocentric robot would name its arms.

The API for dual arm motion planning is very similar to the single arm API, expect that you can supply two joint configurations or two tcp poses instead of one. 

## 2. Scenarios 🖼️

In this section we demonstrate the `airo-planner` API by going over the main Cloth Competition use cases.

TODO:
- scene without cloth obstacle go home
- scene grasp highest (plan to single TCP pose)
- scene with cloth obstacle grasp single arm and filter -> pregrasp no collision, grasp can collide
- scene go to stretch poses (plan to dual TCP poses)

### 2.1 Joints to joints (🦾,🦾) -> (🦾,🦾)

In [ ]:
tangled_joints_left = np.deg2rad([0, -90, -90, -90, 90, 0])
tangled_joints_right = np.deg2rad([-136, -116, -110, -133, 40, 0])
tangled_joints = np.concatenate([tangled_joints_left, tangled_joints_right])

plant.SetPositions(plant_context, tangled_joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
path = planner.plan_to_joint_configuration(tangled_joints_left, tangled_joints_right, home_joints_left, home_joints_right)
print(path.shape)

In [ ]:
from airo_drake import time_parametrize_toppra, animate_dual_joint_trajectory

trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)

animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)

### 2.2 Joints to single arm joints (🦾,🦾) -> (🦾,💤)

Note that you are allow to set one of the goals to `None` to indicate that that arm should not move.

In [ ]:
path = planner.plan_to_joint_configuration(tangled_joints_left, tangled_joints_right, home_joints_left, None)

In [ ]:
trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory)

### 2.3 Joints to single TCP pose  (🦾,🦾) -> (💤,📐)

In [ ]:
from airo_drake import animate_joint_configurations

grasp_pose = np.identity(4)
grasp_pose[2, 3] = 0.4

grasp_joints = inverse_kinematics_right_fn(grasp_pose)

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_right_index, grasp_joints, context=context)

In [ ]:
path = planner.plan_to_tcp_pose(tangled_joints_left, tangled_joints_right, None, grasp_pose)